Fake News Detection

Dataset Source: Kaggle

About the Dataset:

1. id: Unique id for a news article

2. title: Title of the news article

3. author: Author of the news article

4. text: The content in the article (could be incomplete)

5. label: 1 for Fake news and 0 for Real news.

In [4]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [5]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\91979\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [6]:
#printing the stopwords in English
print(stopwords.words('english'))


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [15]:
# loading the dataset to a pandas DataFrame
news_dataset = pd.read_csv("/Users/91979/Desktop/train.csv")

In [16]:
news_dataset.shape

(20800, 5)

In [17]:
# print the first 5 rows of the dataframe
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [18]:
# counting the number of missing values in the dataset
news_dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [19]:
# replacing the null values with empty string
news_dataset = news_dataset.fillna('')

In [20]:
# merging the author name and news title
news_dataset['content'] = news_dataset['author']+' '+news_dataset['title']

In [21]:
print(news_dataset['content'])

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


In [22]:
# separating the data & label
X = news_dataset.drop(columns='label', axis=1)
Y = news_dataset['label']

In [23]:
print(X)
print(Y)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

In [24]:
port_stem = PorterStemmer()

In [25]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-A]',' ', content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content

In [ ]:
news_dataset['content'] = news_dataset['content'].apply(stemming)

In [35]:
print(news_dataset['content'])

0        arrel ucu ou em aid e idn ven ee omey etter nt...
1           aniel lynn illari linton ig oman ampu reitbart
2                  onsortiumnew com hy ruth ight et ou ire
3        essica urkiss ivilian ill n ingl airstrik ave ...
4        oward ortnoy ranian woman jail fiction unpubli...
                               ...                        
20795       erom udson apper rump oster hild hite upremaci
20796    enjamin offman layoff chedul atchup dd ew ork ime
20797    ichael de la erc achel abram aci aid eceiv ake...
20798          alex ansari ussia old arallel xerci n alkan
20799                             avid wanson hat eep aliv
Name: content, Length: 20800, dtype: object


In [36]:
# separating the data and label
X = news_dataset['content'].values
Y = news_dataset['label'].values

In [37]:
print(X)

['arrel ucu ou em aid e idn ven ee omey etter ntil ason haffetz weet'
 'aniel lynn illari linton ig oman ampu reitbart'
 'onsortiumnew com hy ruth ight et ou ire' ...
 'ichael de la erc achel abram aci aid eceiv akeov approach udson ay ew ork ime'
 'alex ansari ussia old arallel xerci n alkan' 'avid wanson hat eep aliv']


In [38]:
print(Y)

[1 0 1 ... 0 1 1]


In [39]:
Y.shape

(20800,)

In [40]:
# converting the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [41]:
print(X)

  (0, 15946)	0.27689250572348606
  (0, 15729)	0.2588776691448143
  (0, 14855)	0.3330968594922744
  (0, 11604)	0.16824447247502153
  (0, 10762)	0.23011147650297142
  (0, 10140)	0.3041908330084778
  (0, 7320)	0.2801763370860095
  (0, 6449)	0.3490653572864078
  (0, 5803)	0.2550787589129956
  (0, 4934)	0.2507937583229387
  (0, 4483)	0.21288260998527464
  (0, 2248)	0.2203743402433526
  (0, 2042)	0.319379440735689
  (0, 582)	0.21765718285022764
  (1, 12861)	0.20072140960955565
  (1, 10707)	0.3937031194892832
  (1, 9116)	0.46186557683760604
  (1, 8872)	0.24940886816964525
  (1, 7571)	0.24802037922546324
  (1, 7400)	0.353252580852867
  (1, 1378)	0.34060738574297966
  (1, 1186)	0.4763079772862734
  (2, 13597)	0.3892203012834804
  (2, 11604)	0.28918316988795734
  (2, 10999)	0.4635884956336797
  :	:
  (20797, 7645)	0.08687477871610777
  (20797, 7213)	0.18590540602845373
  (20797, 5899)	0.08192564565928616
  (20797, 5362)	0.3126755676185308
  (20797, 4323)	0.2941707335026196
  (20797, 3801)	0.2291

In [42]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)


In [43]:
model = LogisticRegression()

In [44]:
model.fit(X_train, Y_train)

LogisticRegression()

In [45]:
# accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [46]:
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.983233173076923


In [47]:
# accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [48]:
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the test data :  0.9747596153846154


In [ ]:
X_new = X_test[1]

prediction = model.predict(X_new)
print(prediction)

if(prediction[0]==0):
  print('The news is Real')
else:
  print('The news is Fake')

[0]
The news is Real


In [57]:
print(Y_test[1])

0
